In [35]:
!pip install fuzzysearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 238.1 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for fuzzysearch: filename=fuzzysearch-0.7.3-py3-none-any.whl size=21203 sha256=1763db4408b6e75ad610aac9fec74e639166ee25b82269ab63a2a18533490ef3
  Stored in directory: /tmp/pip-ephem-wheel-cache-pc3h34mr/wheels/be/ad/2e/bd664c4b01e5535ee4387d8a491311f61467a43627597684a7
Successfully built fuzzysearch


In [37]:
from rapidfuzz import process, fuzz
import re
from fuzzysearch import find_near_matches


In [73]:


class PhraseCorrectorByWords:
    def __init__(self, words_file="words.csv",score_cutoff=70, min_len = 5):
        with open(words_file, "r", encoding="utf-8") as f:
            self.reference_words = [line.strip() for line in f]
        self.score_cutoff = score_cutoff
        self.min_len = min_len





    def correct_text(self,text):
        corrected_lines = []
        corrections_log = []
        for line_num, line in enumerate(text.strip().split('\n')):
            corrected_words = []
            words = re.split(r"[ \t\f\v.,!?;:()\"«»—–]+", line.upper().replace("Ё", "Е"))
            words = list(filter(None, words))  # убираем пустые строки, если они есть
        
            for word_pos, word in enumerate(words):
                if len(word) < self.min_len:
                    corrected_words.append(word)
                    continue
                # Ищем наиболее близкое слово из словаря
                match = process.extractOne(word, self.reference_words, scorer=fuzz.ratio, score_cutoff=self.score_cutoff)
                #print(match)
                corrected_word = match[0] if match else word
                corrected_words.append(corrected_word)
                if match and corrected_word != word:
                    corrections_log.append({
                        "line": line_num + 1,
                        "position": word_pos + 1,
                        "original": word,
                        "corrected": corrected_word,
                        "score": match[1]
                    })
            corrected_lines.append(' '.join(corrected_words))
        return '\n'.join(corrected_lines), corrections_log
    


In [156]:

class PhraseCorrectorNgrams:
    def __init__(self, words_file="words.csv", score_cutoff=85, min_len=3, max_ngram=4):
        with open(words_file, "r", encoding="utf-8") as f:
            self.reference_phrases = [line.strip().upper().replace("Ё", "Е") for line in f]
        self.score_cutoff = score_cutoff
        self.min_len = min_len
        self.max_ngram = max_ngram

    def correct_text(self, text):
        text = text.upper().replace("Ё", "Е")
        tokens = re.split(r"[ \n\t\f\v!?;:()\"«»]+", text)
        #print(tokens)
        tokens = list(filter(None, tokens))

        corrected_tokens = tokens[:]
        corrected = [False] * len(tokens)
        corrections_log = []

        for n in range(self.max_ngram, 0, -1):
            for i in range(len(tokens) - n + 1):
                if any(corrected[i:i+n]):
                    continue

                ngram_tokens = tokens[i:i+n]
                ngram = ' '.join(ngram_tokens)
                if len(ngram) < self.min_len:
                    continue

                match = process.extractOne(ngram, self.reference_phrases, scorer=fuzz.ratio, score_cutoff=self.score_cutoff)
                if match:
                    corrected_tokens[i] = match[0]
                    for j in range(i+1, i+n):
                        corrected_tokens[j] = ''
                    corrected[i:i+n] = [True]*n

                    corrections_log.append({
                        "position": i,
                        "original": ngram,
                        "corrected": match[0],
                        "score": match[1],
                    })

        final_text = ' '.join(filter(None, corrected_tokens))
        return final_text, corrections_log

In [88]:
from fuzzysearch import find_near_matches

class PhraseCorrectorPuzzySearch:
    def __init__(self, words_file="words.csv", max_l_dist=1, min_len=5):
        with open(words_file, "r", encoding="utf-8") as f:
            self.reference_phrases = [line.strip().upper().replace("Ё", "Е") for line in f]
        self.max_l_dist = max_l_dist
        self.min_len = min_len

    def correct_text(self, text):
        text = text.upper().replace("Ё", "Е")
        corrected_text = text
        corrections_log = []

        for phrase in self.reference_phrases:
            if len(phrase) < self.min_len:
                continue

            matches = find_near_matches(phrase, corrected_text, max_l_dist=self.max_l_dist)
            for match in matches:
                original_segment = corrected_text[match.start:match.end]
                corrected_text = (
                    corrected_text[:match.start] + phrase + corrected_text[match.end:]
                )

                corrections_log.append({
                    "original": original_segment,
                    "corrected": phrase,
                    "start_pos": match.start,
                    "end_pos": match.end,
                    "dist": match.dist
                })

        return corrected_text, corrections_log


In [129]:
phrase_corrector = PhraseCorrectorNgrams(words_file="words.txt", score_cutoff=75,  min_len=2)

In [130]:
to_be_corrected = '''
Пюре Фрутонята
Цыпленок 80г
Мясое изделие для детского питания
'''


corrected_text, corrected_log = phrase_corrector.correct_text(to_be_corrected)
print(corrected_text)
print("\nЛог замен:")
for entry in corrected_log:
    print(entry)

['', 'ПЮРЕ', 'ФРУТОНЯТА', 'ЦЫПЛЕНОК', '80Г', 'МЯСОЕ', 'ИЗДЕЛИЕ', 'ДЛЯ', 'ДЕТСКОГО', 'ПИТАНИЯ', '']
ПЮРЕ ФРУТОНЯНЯ ЦЫПЛЕНОК 80Г МЯСНОЕ ИЗДЕЛИЕ ДЛЯ ДЕТСКОГО ПИТАНИЯ

Лог замен:
{'position': 0, 'original': 'ПЮРЕ', 'corrected': 'ПЮРЕ', 'score': 100.0}
{'position': 1, 'original': 'ФРУТОНЯТА', 'corrected': 'ФРУТОНЯНЯ', 'score': 77.77777777777779}
{'position': 2, 'original': 'ЦЫПЛЕНОК', 'corrected': 'ЦЫПЛЕНОК', 'score': 100.0}
{'position': 4, 'original': 'МЯСОЕ', 'corrected': 'МЯСНОЕ', 'score': 90.9090909090909}
{'position': 5, 'original': 'ИЗДЕЛИЕ', 'corrected': 'ИЗДЕЛИЕ', 'score': 100.0}
{'position': 7, 'original': 'ДЕТСКОГО', 'corrected': 'ДЕТСКОГО', 'score': 100.0}
{'position': 8, 'original': 'ПИТАНИЯ', 'corrected': 'ПИТАНИЯ', 'score': 100.0}


In [131]:
to_be_corrected = '''
Пюре фруктоня
цыбленок 80000г
мясное для детского питания
'''

corrected_text = phrase_corrector.correct_text(to_be_corrected)

corrected_text, corrected_log = phrase_corrector.correct_text(to_be_corrected)
print(corrected_text)
print("\nЛог замен:")
for entry in corrected_log:
    print(entry)

['', 'ПЮРЕ', 'ФРУКТОНЯ', 'ЦЫБЛЕНОК', '80000Г', 'МЯСНОЕ', 'ДЛЯ', 'ДЕТСКОГО', 'ПИТАНИЯ', '']
['', 'ПЮРЕ', 'ФРУКТОНЯ', 'ЦЫБЛЕНОК', '80000Г', 'МЯСНОЕ', 'ДЛЯ', 'ДЕТСКОГО', 'ПИТАНИЯ', '']
ПЮРЕ ФРУТОНЯНЯ ЦЫПЛЕНОК 80000Г МЯСНОЕ ДЛЯ ДЕТСКОГО ПИТАНИЯ

Лог замен:
{'position': 0, 'original': 'ПЮРЕ', 'corrected': 'ПЮРЕ', 'score': 100.0}
{'position': 1, 'original': 'ФРУКТОНЯ', 'corrected': 'ФРУТОНЯНЯ', 'score': 82.35294117647058}
{'position': 2, 'original': 'ЦЫБЛЕНОК', 'corrected': 'ЦЫПЛЕНОК', 'score': 87.5}
{'position': 4, 'original': 'МЯСНОЕ', 'corrected': 'МЯСНОЕ', 'score': 100.0}
{'position': 6, 'original': 'ДЕТСКОГО', 'corrected': 'ДЕТСКОГО', 'score': 100.0}
{'position': 7, 'original': 'ПИТАНИЯ', 'corrected': 'ПИТАНИЯ', 'score': 100.0}


In [132]:
to_be_corrected = '''
бальзам ШАУМ 7
ТРАВ 300мл 
для нормальных и жирных волос
'''


corrected_text, corrected_log = phrase_corrector.correct_text(to_be_corrected)
print(corrected_text)
print("\nЛог замен:")
for entry in corrected_log:
    print(entry)

['', 'БАЛЬЗАМ', 'ШАУМ', '7', 'ТРАВ', '300МЛ', 'ДЛЯ', 'НОРМАЛЬНЫХ', 'И', 'ЖИРНЫХ', 'ВОЛОС', '']
БАЛЬЗАМ ШАУМА 7 ТРАВ 300МЛ ДЛЯ НОРМАЛЬНЫХ И ЖИРНЫХ ВОЛОС

Лог замен:
{'position': 0, 'original': 'БАЛЬЗАМ', 'corrected': 'БАЛЬЗАМ', 'score': 100.0}
{'position': 1, 'original': 'ШАУМ', 'corrected': 'ШАУМА', 'score': 88.88888888888889}
{'position': 3, 'original': 'ТРАВ', 'corrected': 'ТРАВ', 'score': 100.0}
{'position': 6, 'original': 'НОРМАЛЬНЫХ', 'corrected': 'НОРМАЛЬНЫХ', 'score': 100.0}
{'position': 8, 'original': 'ЖИРНЫХ', 'corrected': 'ЖИРНЫХ', 'score': 100.0}
{'position': 9, 'original': 'ВОЛОС', 'corrected': 'ВОЛОС', 'score': 100.0}


In [135]:
to_be_corrected = '''
Балъзам ШАУМА 7
ТРАВ 300мл
для нормальных и жирных волос
'''

corrected_text, corrected_log = phrase_corrector.correct_text(to_be_corrected)
print(corrected_text)
print("\nЛог замен:")
for entry in corrected_log:
    print(entry)

['', 'БАЛЪЗАМ', 'ШАУМА', '7', 'ТРАВ', '300МЛ', 'ДЛЯ', 'НОРМАЛЬНЫХ', 'И', 'ЖИРНЫХ', 'ВОЛОС', '']
БАЛЬЗАМ ШАУМА 7 ТРАВ 300МЛ ДЛЯ НОРМАЛЬНЫХ И ЖИРНЫХ ВОЛОС

Лог замен:
{'position': 0, 'original': 'БАЛЪЗАМ', 'corrected': 'БАЛЬЗАМ', 'score': 85.71428571428572}
{'position': 1, 'original': 'ШАУМА', 'corrected': 'ШАУМА', 'score': 100.0}
{'position': 3, 'original': 'ТРАВ', 'corrected': 'ТРАВ', 'score': 100.0}
{'position': 6, 'original': 'НОРМАЛЬНЫХ', 'corrected': 'НОРМАЛЬНЫХ', 'score': 100.0}
{'position': 8, 'original': 'ЖИРНЫХ', 'corrected': 'ЖИРНЫХ', 'score': 100.0}
{'position': 9, 'original': 'ВОЛОС', 'corrected': 'ВОЛОС', 'score': 100.0}


In [136]:

to_be_corrected = '''
Тютюнок Занукелия
500 мг
'''


corrected_text, corrected_log = phrase_corrector.correct_text(to_be_corrected)
print(corrected_text)
print("\nЛог замен:")
for entry in corrected_log:
    print(entry)

['', 'ТЮТЮНОК', 'ЗАНУКЕЛИЯ', '500', 'МГ', '']
ТЮТЮНОК ЗАНУКЕЛИЯ 500 МГ

Лог замен:


In [13]:
import pandas as pd

In [16]:
df_results = pd.read_excel("results_with_original_text.xlsx")

In [19]:
df_results.set_index("file_name", inplace=True)

In [20]:
df_results

,text,elapsed_time,corrected_text,elapsed_time_corrected,Unnamed: 5,Unnamed: 6,Unnamed: 7,original_text
file_name,,,,,,,,
1,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nПригодна пить всем,5.1053,ВОДА СВЯТОЙ ИСТОЧНИК\n1 0Л\nПРИРОДНАЯ ПИТЬ ВСЕМ,0.0004,NaN,NaN,NaN,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nприродная питьевая...
2,"Вода Святой источник\n0,75л\nнегазированная вода",4.2579,ВОДА СВЯТОЙ ИСТОЧНИК\n0 75Л\nНЕГАЗИРОВАННАЯ ВОДА,0.0004,NaN,NaN,NaN,"Вода Святой источник\n0,75л\nнегазированная спорт"
3,Вода ШИШКИН ЛЕС\nпитьевая 1л\nАО «Заирoколина»,5.2103,ВОДА ШИШКИН ЛЕС\nПИТЬЕВАЯ 1Л\nАО ЗАИРOКОЛИНА,0.0006,NaN,NaN,NaN,Вода ШИШКИН ЛЕС\nпитьевая 1л\nнегазированная
4,Цена красная,3.4990,ЦЕНА КРАСНАЯ,0.0001,NaN,NaN,NaN,"Вода КРАСНАЯ ЦЕНА\n1,5л\nнегазированная"
5,Вода Святой Источник\n1.5л\nх.почечен. без сах...,5.0172,ВОДА СВЯТОЙ ИСТОЧНИК\n1 5Л\nХ ПЕЧЕН БЕЗ САХ ПИЛ Б,0.0005,NaN,NaN,NaN,"Вода СВЯТОЙ ИСТОЧНИК\n1,5л\nключевая, без газа..."
...,...,...,...,...,...,...,...,...
1004,Томаты,3.3057,ТОМАТЫ,0.0001,NaN,NaN,NaN,Томаты 1кг\nnan\nnan
1005,Ароматизированный,3.4513,ГЛАЗИРОВАННЫЙ,0.0003,NaN,NaN,NaN,Апельсины 1кг\nnan\nnan
1006,Нектарины\n1кг\nузбекские,4.2026,НЕКТАРИНЫ\n1КГ\nУЗБЕКСКИЕ,0.0005,NaN,NaN,NaN,Нектарины 1кг\nузбекские\nnan


In [21]:
df_results_corrector = df_results.copy()

In [22]:
df_results_corrector = df_results_corrector[["text", "corrected_text", "original_text"]]

In [23]:
df_results_corrector

,text,corrected_text,original_text
file_name,,,
1,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nПригодна пить всем,ВОДА СВЯТОЙ ИСТОЧНИК\n1 0Л\nПРИРОДНАЯ ПИТЬ ВСЕМ,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nприродная питьевая...
2,"Вода Святой источник\n0,75л\nнегазированная вода",ВОДА СВЯТОЙ ИСТОЧНИК\n0 75Л\nНЕГАЗИРОВАННАЯ ВОДА,"Вода Святой источник\n0,75л\nнегазированная спорт"
3,Вода ШИШКИН ЛЕС\nпитьевая 1л\nАО «Заирoколина»,ВОДА ШИШКИН ЛЕС\nПИТЬЕВАЯ 1Л\nАО ЗАИРOКОЛИНА,Вода ШИШКИН ЛЕС\nпитьевая 1л\nнегазированная
4,Цена красная,ЦЕНА КРАСНАЯ,"Вода КРАСНАЯ ЦЕНА\n1,5л\nнегазированная"
5,Вода Святой Источник\n1.5л\nх.почечен. без сах...,ВОДА СВЯТОЙ ИСТОЧНИК\n1 5Л\nХ ПЕЧЕН БЕЗ САХ ПИЛ Б,"Вода СВЯТОЙ ИСТОЧНИК\n1,5л\nключевая, без газа..."
...,...,...,...
1004,Томаты,ТОМАТЫ,Томаты 1кг\nnan\nnan
1005,Ароматизированный,ГЛАЗИРОВАННЫЙ,Апельсины 1кг\nnan\nnan
1006,Нектарины\n1кг\nузбекские,НЕКТАРИНЫ\n1КГ\nУЗБЕКСКИЕ,Нектарины 1кг\nузбекские\nnan


In [140]:
df_results_corrector["log"] = None
df_results_corrector["corrected_text"] = None

In [141]:
df_results_corrector

,text,corrected_text,original_text,log
file_name,,,,
1,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nПригодна пить всем,None,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nприродная питьевая...,None
2,"Вода Святой источник\n0,75л\nнегазированная вода",None,"Вода Святой источник\n0,75л\nнегазированная спорт",None
3,Вода ШИШКИН ЛЕС\nпитьевая 1л\nАО «Заирoколина»,None,Вода ШИШКИН ЛЕС\nпитьевая 1л\nнегазированная,None
4,Цена красная,None,"Вода КРАСНАЯ ЦЕНА\n1,5л\nнегазированная",None
5,Вода Святой Источник\n1.5л\nх.почечен. без сах...,None,"Вода СВЯТОЙ ИСТОЧНИК\n1,5л\nключевая, без газа...",None
...,...,...,...,...
1004,Томаты,None,Томаты 1кг\nnan\nnan,None
1005,Ароматизированный,None,Апельсины 1кг\nnan\nnan,None
1006,Нектарины\n1кг\nузбекские,None,Нектарины 1кг\nузбекские\nnan,None


In [145]:
df_results_corrector[df_results_corrector["text"].isna()]


,text,corrected_text,original_text,log
file_name,,,,
168,NaN,None,-\nnan\nnan,None
169,NaN,None,-\nnan\nnan,None
170,NaN,None,-\nnan\nnan,None
184,NaN,None,-\nnan\nnan,None
284,NaN,None,-\nnan\nnan,None
976,NaN,None,Киви 1кг\nnan\nnan,None


In [143]:
import json

In [159]:
corrector = PhraseCorrectorNgrams("words.txt", score_cutoff=75, min_len=3)
for index, row in df_results_corrector.iterrows():
    text = row["text"]
    if not isinstance(text, str):
        continue  # или можно что-то другое, напр. записать "None"
    
    corrected_text, corrected_log = corrector.correct_text(text)
    df_results.at[index, "corrected_text"] = corrected_text
    df_results.at[index, "log"] = json.dumps(corrected_log, ensure_ascii=False)


In [161]:
df_results[:100]

,text,elapsed_time,corrected_text,elapsed_time_corrected,Unnamed: 5,Unnamed: 6,Unnamed: 7,original_text,log
file_name,,,,,,,,,
1,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nПригодна пить всем,5.1053,ВОДА СВЯТОЙ ИСТОЧНИК ПРИРОДНАЯ ПЯТЬ ВСЕХ,0.0004,NaN,NaN,NaN,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nприродная питьевая...,"[{""position"": 2, ""original"": ""ИСТОЧНИК 1.0Л"", ..."
2,"Вода Святой источник\n0,75л\nнегазированная вода",4.2579,ВОДА СВЯТОЙ ИСТОЧНИК НЕГАЗИРОВАННАЯ ВОДА,0.0004,NaN,NaN,NaN,"Вода Святой источник\n0,75л\nнегазированная спорт","[{""position"": 3, ""original"": ""0,75Л НЕГАЗИРОВА..."
3,Вода ШИШКИН ЛЕС\nпитьевая 1л\nАО «Заирoколина»,5.2103,ВОДА ШИШКИН ПИТЬЕВАЯ АО ЗАИРOКОЛИНА,0.0006,NaN,NaN,NaN,Вода ШИШКИН ЛЕС\nпитьевая 1л\nнегазированная,"[{""position"": 1, ""original"": ""ШИШКИН ЛЕС"", ""co..."
4,Цена красная,3.4990,ЦЕНА КРАСНАЯ,0.0001,NaN,NaN,NaN,"Вода КРАСНАЯ ЦЕНА\n1,5л\nнегазированная","[{""position"": 0, ""original"": ""ЦЕНА"", ""correcte..."
5,Вода Святой Источник\n1.5л\nх.почечен. без сах...,5.0172,ВОДА СВЯТОЙ ИСТОЧНИК Х.ПОЧЕЧЕН. БЕЗЕ САХ. ПИЛ/Б,0.0005,NaN,NaN,NaN,"Вода СВЯТОЙ ИСТОЧНИК\n1,5л\nключевая, без газа...","[{""position"": 2, ""original"": ""ИСТОЧНИК 1.5Л"", ..."
...,...,...,...,...,...,...,...,...,...
96,Водка ПЕТРОВ.\nРЕГЛАМЕНТ 0.25л\n40%,4.8559,ВОДКА ПЕТРОВ. РЕГЛАМЕНТ 40%,0.0004,NaN,NaN,NaN,Водка ПЕТРОВ.\nРЕГЛАМЕНТ 0.25л\n0.4,"[{""position"": 2, ""original"": ""РЕГЛАМЕНТ 0.25Л""..."
97,Водка TUNDRA\n0.25л\nAUTHENTIC 40%,4.4127,ВОДКА TUNDRA AUTHENTIC 40%,0.0004,NaN,NaN,NaN,Водка TUNDRA\n0.25л\nAUTHENTIC 40%,"[{""position"": 2, ""original"": ""0.25Л AUTHENTIC""..."
98,Водка ПЕТРОВ.\nРЕГЛАМЕНТ\n0.25л\n40%,4.8586,ВОДКА ПЕТРОВ. РЕГЛАМЕНТ 40%,0.0004,NaN,NaN,NaN,Водка ПЕТРОВ.\nРЕГЛАМЕНТ 0.25л\n0.4,"[{""position"": 2, ""original"": ""РЕГЛАМЕНТ 0.25Л""..."


In [155]:
df_results.to_excel("results_with_corrected_text.xlsx", index=False)